In [60]:
import matplotlib.pyplot as plt
import numpy as np
import scipy
import helper_functions as hf
from data import interpolate_temperature

In [63]:
instrument = '100MHz'
year = '2021'

channel = 'EW' # curently must be opposite of what you want.

path2file = f'../Data/{year}/{instrument[:-3]}/{channel}/'
file_ending = f'_{year}_{instrument[:-3]}{channel}.npy'

with open(path2file + 'res50' + file_ending, 'rb') as f:
    P50 = np.load(f)
with open(path2file + 'res100' + file_ending, 'rb') as f:
    P100 = np.load(f)
with open(path2file + 'short' + file_ending, 'rb') as f:
    Pshort = np.load(f)
with open(path2file + 'systime' + file_ending, 'rb') as f:
    systime = np.load(f)

channel = 'NS'
    
# rfi flagged data & freq
with open(f'RFI_flagged/rfi_flagged_{instrument[:-3]}{channel}.npz', 'rb') as f:
    data_file = np.load(f)
    data = data_file['data'] ####### need to use short only calib. this is res50 calib
    freq = data_file['freq']

In [64]:
# upload ambient temperature

ambient_temperature = np.load(f'Temperature/T_ambient_{instrument}.npz')
T_amb, T_amb_time = ambient_temperature['T_amb'], ambient_temperature['systime']

T_amb, systime, interp_inds = interpolate_temperature(T_amb, T_amb_time, systime)

# apply temperature interpolation cuts in time antenna
P50 = P50[interp_inds]
P100 = P100[interp_inds]
Pshort = Pshort[interp_inds]
data = data[interp_inds]

In [ ]:
###### upload eta
eta_ant = []
eta50 = []
eta100 = []

In [59]:
def K_JNC(Z, P_short, P_50, P_100, eta_50, eta_100, T_amb, delta_t=6.4, delta_nu=250/4096): #### check actual integration time. seconds or hours?
    '''
    Parameters:
        delta_nu (float): frequency channel width, MHz
        delta_t (float): integration time, seconds or h ? TODO
    '''
    if Z == 50:
        eta_Z = eta_50
        P_load = P_50
    elif Z == 100:
        eta_Z = eta_100
        P_load = P_100
    else:
        print('Z (impedence) must be 50 or 100')
        
    P_Z = (P_50 - P_100) / (eta_50 - 4 * eta_100) * Z**2 * eta_Z
    
    return T_amb / (P_load - P_short - P_Z - P_thermal(P_load, delta_t=delta_t, delta_nu=delta_nu))
    

def P_thermal(P_tot, delta_t=6.4, delta_nu=250/4096):
    delta_nu *= 1e6 # convert MHz to Hz
    return P_tot / np.sqrt(delta_nu * delta_t)

In [ ]:
Z = 50
integration_time = np.median(np.diff(systime))
freq_width = 1

K = K_JNC(Z, Pshort, P50, P100, eta50, eta100, delta_t=integration_time, delta_nu=freq_width)

In [ ]:
# crop K to match antenna power data
raw_freq = np.arange(0, 250, 250/4096)
highpass = 49.5
lowpass = 135.5
K, _ = hf.truncate(K, raw_freq, highpass, lowpass)

In [ ]:
Tsky = K * (data - P_thermal(data, delta_t=integration_time, delta_nu=freq_width)) / eta_ant

### Compare to GSM

In [ ]:
Tgsm = np.load(f'GSM_averages/{instrument}_{channel}_GSM_average_2minbins_horizon_0align.npy')
gsm_flow = 30

In [ ]:
# bin data to compare with Tgsm
binsize_f = 1
binsize_t = 2
flow = 50
fhigh = 135
Tsky_binned, freq_1mhz, lst_2min, bin_inds_f, bin_inds_t = hf.binning(Tsky, freq, lst, binsize_f, binsize_t, flow, fhigh)

In [ ]:
f = 90
offset = 10

fig = plt.figure()
ax = fig.subplots(1,2)
ax[0].grid()
ax[0].set_title(f'{f}MHz')
ax[0].set_ylabel('Temperature (K)')
ax[0].set_xlabel('LST')
ax[0].plot(lst_2min, Tsky_binned[:, (f - flow)], label='data')
ax[0].plot(lst_2min, Tgsm[(np.arange(720)+offset)%720, (f - gsm_flow)//2], label='gsm')
ax[0].legend()

ax[1].grid()
ax[1].set_ylabel('residual (K)')
ax[1].set_xlabel('LST')
ax[1].plot(lst_2min, Tsky_binned[:, (f - flow)] - Tgsm[(np.arange(720)+offset)%720, (f - gsm_flow)//2])

In [ ]:
plt.figure()
plt.xlabel('Frequency (MHz)')
plt.ylabel('LST (hours)')
plt.pcolormesh(freq_1mhz, lst_2min, Tsky_binned - Tgsm[(np.arange(720)+offset)%720], cmap='jet', shading='nearest')
plt.colorbar(label='res (K)')
plt.show()